##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

WE, THE DELIGHTFUL FROGS, BASED THE FOLLOWING CODE OFF TENSORFLOW'S TUTORIAL HERE:

https://www.tensorflow.org/text/tutorials/text_generation

# Text generation with an RNN

## Setting up the hyperparameters

If you want to change the model used, see section 'Build the Model'

In [ ]:
path_to_input = 'scripts_all.txt'
path_to_output = 'all_e30_s30.txt'
model_save_file = 'one_step_all_e30_s30'
checkpoint_dir = './training_checkpoints_all_e30_s30'

seq_length = 30     # Predict the next char based on the last x char

embedding_dim = 256 # The embedding dimension
rnn_units = 1024    # Number of RNN units

# Batch size
BATCH_SIZE = 64

# How many times we go over the training data
EPOCHS = 30

# The starting data to predict on
start_chars = 'FROGS:'
# How many thousand char to predict
kthousand = 12

## Importing TensorFlow and other libraries

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

## Reading in the data

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_input, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')

Length of text: 871612 characters


In [ ]:
# Peak at the first 250 characters
print(text[:250])

ALOIS:
Darkness... Darkness... Darkness. Darkness is coiling itself around me... I am ensnared by the darkness, and every last drop of my blood will be drained away by the blade piercing me. 

CLAUDE:
That said... 

ALOIS:
That said, I still want thi


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
vocab_size = len(vocab)
print(f'{vocab_size} unique characters')

91 unique characters


## Processing the text

### Vectorizing the text

Before training, we need the chars in a numerical representation. 

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

And we need to do the reverse, which is going from ids to characters. Also for easier human reading,we want to be able to go from characters to a string of text.

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

A
L
O
I
S
:


D
a
r


### Creating training examples and targets

We need to divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

The `batch` method easily converts these individual characters to sequences of the desired  size.

In [ ]:
examples_per_epoch = len(text)//(seq_length+1)

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right (so that it knows what the next char should be).


In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'ALOIS:\nDarkness... Darkness...'
Target: b'LOIS:\nDarkness... Darkness... '


### Creating training batches

We need to shuffle the sequences and pack it into batches.

In [ ]:
# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 30), dtype=tf.int64, name=None), TensorSpec(shape=(64, 30), dtype=tf.int64, name=None))>

## Building The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

If we wanted, we could change the model to be any series of layers that we want with other possibilities being RNN, LSTM, alternating layers, etc. Then using keras.Sequential would be the method to use for the construction of the model.

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.algorithm = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.algorithm.get_initial_state(x)
    x, states = self.algorithm(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

model.summary()

(64, 30, 92) # (batch_size, sequence_length, vocab_size)
Model: "my_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     multiple                  23552     
                                                                 
 gru_4 (GRU)                 multiple                  3938304   
                                                                 
 dense_4 (Dense)             multiple                  94300     
                                                                 
Total params: 4,056,156
Trainable params: 4,056,156
Non-trainable params: 0
_________________________________________________________________


## Training the model

unmodified from the tutorial

### Attaching an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 30, 92)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.5221605, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [ ]:
tf.exp(example_batch_mean_loss).numpy()

92.034225

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Configuring save checkpoints for the model

In [ ]:
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

###  Training the model

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
439/439 [==============================] - 10s 16ms/step - loss: 2.1711
Epoch 2/30
439/439 [==============================] - 8s 18ms/step - loss: 1.5715
Epoch 3/30
439/439 [==============================] - 8s 17ms/step - loss: 1.3924
Epoch 4/30
439/439 [==============================] - 8s 16ms/step - loss: 1.2967
Epoch 5/30
439/439 [==============================] - 8s 17ms/step - loss: 1.2254
Epoch 6/30
439/439 [==============================] - 8s 17ms/step - loss: 1.1614
Epoch 7/30
439/439 [==============================] - 8s 17ms/step - loss: 1.1003
Epoch 8/30
439/439 [==============================] - 8s 17ms/step - loss: 1.0403
Epoch 9/30
439/439 [==============================] - 8s 17ms/step - loss: 0.9811
Epoch 10/30
439/439 [==============================] - 8s 17ms/step - loss: 0.9228
Epoch 11/30
439/439 [==============================] - 8s 17ms/step - loss: 0.8691
Epoch 12/30
439/439 [==============================] - 8s 18ms/step - loss: 0.8195
Epoch 13/30


## Generating text

The following is to generate a 1 character at a time and using that generated character to make the ones that follow. 

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Now telling it to actually do that generation.

In [ ]:

start = time.time()
states = None
start_chars='FROGS:'
next_char = tf.constant([start_chars])
filewrite = open(path_to_output, 'a')

for i in range(kthousand+1):
    result = [next_char]

    for n in range(1000): 
      next_char, states = one_step_model.generate_one_step(next_char, states=states)
      result.append(next_char)

      if i==kthousand and next_char=='.':
          break
    #if i==kthousand

    result = tf.strings.join(result)

    a = result[0].numpy().decode('utf-8')
    filewrite.write(a)
    next_char = tf.constant([a[-10:]])

filewrite.close()

end = time.time()
#print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)



Run time: 28.6715350151062


## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [ ]:
tf.saved_model.save(one_step_model, model_save_file)

INFO:tensorflow:Assets written to: one_step_all_e30_s30/assets


INFO:tensorflow:Assets written to: one_step_all_e30_s30/assets


In [ ]:
"""
path_to_output = 'action_e30_s50.txt'
one_step_reloaded = tf.saved_model.load('one_step_action_e30_s50')

start = time.time()
states = None
start_chars='FROGS:'
next_char = tf.constant([start_chars])
filewrite = open(path_to_output, 'a')

for i in range(kthousand+1):
    result = [next_char]

    for n in range(1000): 
      next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
      result.append(next_char)

      if i==kthousand and next_char=='.':
          break

    result = tf.strings.join(result)

    a = result[0].numpy().decode('utf-8')
    filewrite.write(a)
    next_char = tf.constant([a[-10:]])

filewrite.close()

end = time.time()
#print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)
"""

"\npath_to_output = 'action_e30_s50.txt'\none_step_reloaded = tf.saved_model.load('one_step_action_e30_s50')\n\nstart = time.time()\nstates = None\nstart_chars='FROGS:'\nnext_char = tf.constant([start_chars])\nfilewrite = open(path_to_output, 'a')\n\nfor i in range(kthousand+1):\n    result = [next_char]\n\n    for n in range(1000): \n      next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)\n      result.append(next_char)\n\n      if i==kthousand and next_char=='.':\n          break\n\n    result = tf.strings.join(result)\n\n    a = result[0].numpy().decode('utf-8')\n    filewrite.write(a)\n    next_char = tf.constant([a[-10:]])\n\nfilewrite.close()\n\nend = time.time()\n#print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)\nprint('\nRun time:', end - start)\n"